### 引言

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [6]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand([2, 20])
net(X)

tensor([[ 0.0857,  0.2138, -0.2470,  0.0100, -0.2079, -0.3314,  0.0655, -0.1274,
         -0.0880, -0.0825],
        [ 0.0036,  0.4161, -0.1498, -0.0766, -0.3779, -0.3463,  0.1470, -0.0300,
         -0.0629, -0.0663]], grad_fn=<AddmmBackward0>)

### 5.1.1 自定义块

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.output = nn.Linear(256, 10)  # 输出层

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

In [12]:
net = MLP()
net(X)

tensor([[ 0.0705, -0.0159,  0.1194,  0.0948, -0.0037,  0.1555, -0.0764,  0.1016,
          0.2273,  0.0761],
        [ 0.0681, -0.0539,  0.1736,  0.1189,  0.0714,  0.0718, -0.1393,  0.1422,
          0.2123,  0.0449]], grad_fn=<AddmmBackward0>)

### 5.1.2 顺序块

In [13]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()

        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [14]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0038,  0.0884,  0.1734, -0.1875, -0.0113,  0.0055, -0.0905, -0.0526,
          0.2543,  0.0690],
        [-0.0035,  0.0885,  0.1544, -0.1683,  0.0276, -0.0018, -0.0823,  0.0289,
          0.2941,  0.1199]], grad_fn=<AddmmBackward0>)

### 5.1.3 在前向传播中执行代码

In [19]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()

        # 不计算梯度的固定随机参数
        self.rand_weight = torch.rand([20, 20], requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用常量参数及relu, mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层
        X = self.linear(X)

        while X.abs().sum() > 1:  # L1范数
            X /= 2
        return X.sum()

In [21]:
net = FixedHiddenMLP()
net(X)

tensor(0.0417, grad_fn=<SumBackward0>)

In [22]:
# 混合搭配各种组合块
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

In [23]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1530, grad_fn=<SumBackward0>)